In [16]:
# 브루트 포싱 역전 카운팅 알고리즘.
def inverse_count(lst):
    count = 0

    # 모든 가능한 조합에 대해서 수행.
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            # 두 원소가 역전된 관계이면 count++
            if i != j and lst[i] > lst[j]:
                count += 1
    return count

In [17]:
# 분할정복 역전 카운팅 알고리즘을 설계하기 위한 분할정복 정렬 알고리즘 설계.

# 병합정렬의 인수 단순화를 위한 함수.
def msort(lst):
    __msort(lst, 0, len(lst) - 1)

# 병합정렬 주요 로직.
def __msort(lst, left, right):
    # 전달된 서브 리스트의 범위가 유효할 때 수행.
    if left < right:
        middle = (left + right) // 2                # 리스트의 중간 인덱스.
        __msort(lst, left, middle)                  # 좌측 서브 리스트에 대해 재귀적으로 수행.
        __msort(lst, middle + 1, right)             # 우측 서브 리스트에 대해 재귀적으로 수행.
        __msort_merge(lst, left, right, middle)     # 좌측 서브 리스트와 우측 서브 리스트 병합.

def __msort_merge(lst, left, right, middle):
    # 사용되는 인덱스 포인터 변수 설정.
    l = left            # 좌측 서브 리스트
    r = middle + 1      # 우측 서브 리스트

    # 정렬된 배열을 저장할 보조 리스트.
    sorted = []

    # 죄측 인덱스와 우측 인덱스 모두 유효할 때 수행.
    while l <= middle and r <= right:
        # 최측 인덱스가 적절한 원소를 갖고 있으면 이를 보조 리스트에 복사.
        if lst[l] < lst[r]:
            sorted.append(lst[l])
            l += 1

        # 우측 인덱스가 적절한 원소를 갖고 있으면 이를 보조 리스트에 복사.
        else:
            sorted.append(lst[r])
            r += 1

    # 남은 서브 리스트에 잔존하는 원소 복사.
    if (l > middle):
        while (r <= right):
            sorted.append(lst[r])
            r += 1
    else:
        while (l <= middle):
            sorted.append(lst[l])
            l += 1

    # 정렬된 보조 리스트를 메인 리스트에 반영.
    lst[left:right + 1] = sorted

In [18]:
# 분할정복 정렬 알고리즘을 이용한 분할정복 역전 카운팅 알고리즘.
def msort_inverse_count(lst):
    return __msort_inverse_count(lst.copy(), 0, len(lst) - 1)    # 역전된 원소의 개수를 반환하므로 return 사용.

def __msort_inverse_count(lst, left, right):
    if left < right:
        middle = (left + right) // 2
        left_count = __msort_inverse_count(lst, left, middle)
        right_count = __msort_inverse_count(lst, middle + 1, right)
        merge_count = __msort_inverse_count_merge(lst, left, right, middle)
        return left_count + right_count + merge_count            # 서브 리스트의 역전 원소의 개수까지 취합하여 반환.

    # 범위가 올바르지 않으면 역전된 원소를 정의할 수 없으므로 0 반환.
    return 0

def __msort_inverse_count_merge(lst, left, right, middle):
    l = left
    r = middle + 1

    count = 0

    sorted = []
    while l <= middle and r <= right:
        # 정렬된 상태라면 죄측 서브 리스트의 현재 원소가 우특 서브 리스트의 현재 원소보다 작아야 함.
        # 따라서 우측 서브 리스트 원소의 크기가 작으면 역전 원소의 개수 증가.
        if lst[l] <= lst[r]:
            sorted.append(lst[l])
            l += 1
        else:
            sorted.append(lst[r])
            r += 1

            # 양 서브 리스트는 정렬된 상태이므로 좌측 서브 리스트의 모든 원소가 우측 서브 리스트의 현재 원소보다 크다는 의미.
            # 우측 서브 리스트의 현재 원소는 좌측 서브 리스트의 모든 원소에 대해 역전됨.
            count += middle - l + 1 

    if (l > middle):
        while (r <= right):
            sorted.append(lst[r])
            r += 1
    else:
        while (l <= middle):
            sorted.append(lst[l])
            l += 1

    lst[left:right + 1] = sorted
    return count

In [23]:
import random

def main():
    arr = [i for i in range(10)] * 2
    arr.sort(key = lambda e: random.random() < 0.5)

    print(arr)
    print(inverse_count(arr))
    print(msort_inverse_count(arr))

In [24]:
if __name__ == '__main__':
    main()

[0, 1, 3, 5, 6, 9, 1, 3, 5, 9, 2, 4, 7, 8, 0, 2, 4, 6, 7, 8]
64
64
